In [1]:
import pandas as pd
import numpy as np
import bus_bounds_data as bbd
import helper_functions as hp

In [2]:
X_train, X_test, y_train, y_test, df, slack_bus_data, scaler = hp.pre_process_data_from_file()

In [3]:
X_test = hp.unscale_X_test_data(X_test, scaler)

In [4]:
train_df = pd.read_csv("../data_preprocessing/case118_data3.csv", index_col=0)
train_df.reset_index(drop=True, inplace=True)

In [5]:
test_df = pd.read_csv("../predicted_case118_data3.csv", index_col=0)
test_df.reset_index(drop=True, inplace=True)

In [6]:
train_df['time_to_solve'].mean()

0.3067982361364365

In [7]:
test_df['time_to_solve'].mean()

0.2106067347136792

In [8]:
idx_arr = []
for idx in range(len(X_test)):
    row_idx = train_df[(train_df['p_load1'] == round(X_test[idx][0])) & (train_df['p_load2'] == round(X_test[idx][2])) &
                       (train_df['p_load3'] == round(X_test[idx][4])) & (train_df['p_load4'] == round(X_test[idx][6]))].index[0]
    
    if row_idx == None:
        print("Row not found in {} position".format(idx))

    idx_arr.append(row_idx)

In [9]:
X_test_tc = train_df.iloc[idx_arr]['total_cost']
X_test_tts = train_df.iloc[idx_arr]['time_to_solve']

In [10]:
def speedup_factor(test_tts, pred_tts):
    return sum(test_tts / pred_tts) / len(test_tts)

In [11]:
speedup_factor(X_test_tts.values, test_df['time_to_solve'].values)

1.4577658076835815

In [12]:
from egret.parsers.matpower_parser import create_ModelData

In [13]:
model_data = create_ModelData("../data_generation/case118_data/case118.txt")

In [14]:
total_costs = []
for row in range(len(test_df)):
    gen_costs = []
    for idx, p_gen in zip(range(54), test_df.iloc[row, :54].values):
        gen_costs.append(sum(v*p_gen**i for i, v in model_data.data['elements']['generator'][str(idx + 1)]['p_cost']['values'].items()))

    total_costs.append(sum(gen_costs))

In [15]:
def optimality(test_tc, pred_tc):
    return sum((pred_tc - test_tc) / test_tc) / len(test_tc)

In [16]:
optimality(X_test_tc.values, total_costs)

0.21138939970680867

In [17]:
def q_gen_infeasibility(data, num_of_gens):
    q_gen_cols = ["q_gen{}".format(i + 1) for i in range(num_of_gens)]

    infeasibility_df = pd.DataFrame(columns=q_gen_cols)

    for col, key in zip(q_gen_cols, bbd.q_bounds.keys()):
        infeasibility_df[col] = data[col].apply(lambda q_gen : max(bbd.q_bounds[key]["q_min"] - q_gen, 0) + max(q_gen - bbd.q_bounds[key]["q_max"], 0))

    infeasibility_norms = infeasibility_df.apply(lambda x : np.linalg.norm(x), axis=1).values
    
    return sum(infeasibility_norms / len(q_gen_cols)) / len(infeasibility_norms)

In [18]:
q_gen_infeasibility(test_df, 54)

2.604099833957622

In [19]:
Y_bus = hp.get_Ybus_Matrix(118, return_mag_ang=False)

In [20]:
def feasibility(data, Y_bus, num_of_busses):
    sums_of_balances = []
    for i in range(num_of_busses):
        power_balance = []
        for j in range(num_of_busses):
            Vi = data['v_at_bus{}'.format(i + 1)].values
            Vj = data['v_at_bus{}'.format(j + 1)].values
            Gij = np.real(Y_bus[i][j])
            Bij = np.imag(Y_bus[i][j])
            Aij = data['va_at_bus{}'.format(i + 1)].values - data['va_at_bus{}'.format(j + 1)].values

            power_balance.append(Vi * Vj * ((Gij * np.cos(Aij)) + (Bij * np.sin(Aij))))

        sums_of_balances.append(sum(power_balance))

    feasibility_norms = np.linalg.norm(sums_of_balances, axis=0)

    return max(feasibility_norms)


In [21]:
feasibility(test_df, Y_bus, 118)

405.2771396006099